<a href="https://colab.research.google.com/github/anshisboss/Sentence-Transformer/blob/main/Faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords

data = "file.csv"
col = "Description"

# Read data into DataFrame
df = pd.read_csv(data)

# Clean and preprocess the "Description" column
def clean_description(text):


    # Convert to lowercase
    text = text.lower()

    # tokenize the text
    tokens = nltk.word_tokenize(text)

    # remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # rejoin tokens into a string
    text = " ".join(tokens)

    # Remove punctuation and special characters
    text = re.sub(r"[^\w\s]", "", text)

    # Remove extra whitespace
    text = re.sub(r"\s+", " ", text).strip()

    return text

df[col] = df[col].apply(clean_description)

# Show the final shape
print(f"Dataframe shape after cleaning: {df.shape}")
print(df[col])

Dataframe shape after cleaning: (12491, 8)
0        black grey printed medium trolley bag secured ...
1        beige grey made measure kurta churidar dupatta...
2        pink coloured wash 5pocket highrise cropped je...
3        blue selfdesign bandhgala suitblue selfdesign ...
4        brown offwhite printed casual shirt spread col...
                               ...                        
12486    black dark wash 5pocket lowrise jeans clean lo...
12487    pair goldtoned open toe heels regular styling ...
12488    navy blue white printed midrise denim shorts 5...
12489    bvlgari men aqva pour homme marine eau de toil...
12490    black grey striped tshirt polo collar short sl...
Name: Description, Length: 12491, dtype: object


Preprocessing the data


In [7]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.4 MB/s eta 0:00:00


In [8]:
from sentence_transformers import SentenceTransformer

# Load the pre-trained SentenceTransformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Sentences to encode
sentences = df[col]

embeddings = model.encode(sentences)

print(embeddings.shape)

def dot_product(a, b):

    return sum(a_i * b_i for a_i, b_i in zip(a, b))

# for i in range(len(sentences)):
#     for j in range(i + 1, len(sentences)):
#         similarity = dot_product(embeddings[i], embeddings[j])
#         print(f"Similarity between '{sentences[i]}' and '{sentences[j]}': {similarity:.4f}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(12491, 384)


Sentence transformer to generate sentence Embeddings

In [14]:
pip install faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 49.5 MB/s eta 0:00:00


In [19]:
import faiss

d = embeddings.shape[1]

index = faiss.IndexFlatL2(d)

index.is_trained

True

Faiss Library to generate vector indices and Implementing ANN 1. Flat Index that uses centroids and Eucledian distance

In [20]:
index.add(embeddings)

index.ntotal

12491

In [21]:
query = model.encode(['black jeans grey skinny'])

k = 4

Query to check on


In [22]:
%%time
D, arr = index.search(query,k)

for i in arr:
  print(sentences[i])

11542    medium light fade grey jeans skinny fit highri...
11671    light fade grey jeans skinny fit midrise clean...
11386    medium heavy fade grey jeans slim fit midrise ...
11049    dark fade black jeans slim fit midrise clean l...
Name: Description, dtype: object
CPU times: user 8 ms, sys: 1e+03 ns, total: 8 ms
Wall time: 8.52 ms


Knn results using the Flat Index in Faiss

In [23]:
nlist = 50
# number of voronoi cells we want to define
quantizer = faiss.IndexFlatL2(d)
# we need a FlatL2 index for the voronoi cell index to be created
index = faiss.IndexIVFFlat(quantizer,d,nlist)
#check if trained
index.is_trained

False

Voronoi Index formation, it isn't trained

In [24]:
index.train(embeddings)

In [25]:
index.is_trained

True

In [26]:
index.add(embeddings)
index.ntotal

12491

Adding the embeddings to our Voronoi Index

In [27]:
%%time
D, arr = index.search(query,k)

for i in arr:
  print(sentences[i])


11542    medium light fade grey jeans skinny fit highri...
11671    light fade grey jeans skinny fit midrise clean...
11386    medium heavy fade grey jeans slim fit midrise ...
11049    dark fade black jeans slim fit midrise clean l...
Name: Description, dtype: object
CPU times: user 5.5 ms, sys: 25 µs, total: 5.52 ms
Wall time: 5.3 ms


IVFPQ
Product Quantization
Dividing the given vector into SubVectors

In [30]:
# Number of Centroid for Product Quantization
m = 8
bits = 8

quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFPQ(quantizer,d,nlist,m,bits)
#Initializing the index
index.train(embeddings)
#Training and adding the embeddings
index.add(embeddings)

Finally Testing out the PQ index on the same query

In [31]:
%%time
D, arr = index.search(query,k)

for i in arr:
  print(sentences[i])

11542    medium light fade grey jeans skinny fit highri...
11671    light fade grey jeans skinny fit midrise clean...
11386    medium heavy fade grey jeans slim fit midrise ...
11607    dark fade blue jeans skinny fit midrise clean ...
Name: Description, dtype: object
CPU times: user 5.25 ms, sys: 11 µs, total: 5.26 ms
Wall time: 7.49 ms
